In [8]:
## Upload libraries
import sys
import pandas as pd
import numpy as np

#upload HY funcitons
from Functions.calc_func import * 
from Functions.upload_functions import * 

# export file path:
expath = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/'
tests = 'tests/'
exports = 'csv_exports/'

# Canada Province list s loaded.
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
provinces

['CA-NL',
 'CA-PE',
 'CA-NS',
 'CA-NB',
 'CA-QC',
 'CA-ON',
 'CA-MB',
 'CA-SK',
 'CA-AB',
 'CA-BC',
 'CA-YT',
 'CA-NT',
 'CA-NU']

# Load Tables

In [2]:
## IO Tables. (CIRAIG, CanadaIO V2.4, 2022) 2018 Statistics Canada data

# A	The technology matrix (16196 rows × 16196 columns) shows the normalized inter-industry trade. Read the information as "To produce 1$ of the product from the column, I need X$ of the product from the row"																	
															
# Y	The final demand matrix (16196 rows × 3627 columns) shows the demands from the different end-users. Read the information as "The consumer category from the column has bought X$ of the product in the row during the given year"																			
																				
# R	R (104 rows × 6396 columns) gives the factors of production (taxes, value-added, subsidies, etc.). Read the information as "To produce 1$ of the product from the column, I need X$ of factors of production from the row"																			
																				
# S	(11085 rows × 16196 columns) The normalized environmental extensions give the normalized direct emissions for each product. Read the information as "To produce 1$ of the product from the column, I emit Xkg of the pollutant from the row"																			
																				
# FY The direct emissions from the final demand gives the direct emissions from the different end-users, which are thus not directly linked to an economic activity (e.g., burning diesel in one's car). Read the information as "The consumer category from the column has directly emitted Xkg of pollutant from the row during the given year"																			
																				
# C	(33 rows × 11085 columns) The characterization matrix converts emissions of pollutants to impacts on the environment. Read the information as "The pollutant from the column has an impact of X on the impact category from the row"																			
																				
# L (16196 rows × 16196 columns) Liontief matrix

# FY (11085 rows × 3627 columns) - Direct emissions

In [2]:
#This cell loads all od CIRAIG's IO tables
# Tables use 2018 Stat Can. IO data, and openIO V2.4 EEIO model

#load Y table - final demand matrix (16196 rows × 3627)
#make sure to assign columns as indexes and headars, if not deafault

tables_Y = machine_loc('craig_Yfinal_demand.csv')
# load R matrix (104 rows × 6396 columns)
#tables_R = machine_loc('craig_R_norm_tech_matrix.csv',index_colf = [0,1], headerf = [0,1])
#load A matrix (16196 rows × 16196 columns)
tables_A = machine_loc('craig_A_matrix.csv',index_colf = [0,1], headerf = [0,1])
#load L matrix (16196 rows × 16196 columns)
tables_L = machine_loc('craig_L_matrix.csv',index_colf = [0,1], headerf = [0,1])
#load S matrix (11085 rows × 16196 columns)
tables_S = machine_loc('craig_S_norm_env_flow_matrix.csv',index_colf = [0], headerf = [0,1])
#load C matrix (33 rows × 11085 columns)
tables_C = machine_loc('craig_C_characterization_matrix.csv',index_colf = [0,1], headerf = 0)
#load FY matrix (11085 rows × 3626 columns)
#tables_FY = machine_loc('craig_FY_direct_emissions_matrix.csv',index_colf = [0,1], headerf = [0,1,2])

# calculate (C)(S) table - environmental impact
tables_CS = tables_C.dot(tables_S)
print('Table CS calculated successfuly')

print('\n All tables loaded successfuly')

Couldnt load file on VM
loading craig_Yfinal_demand.csv from hard drive...


craig_Yfinal_demand.csv loaded successfuly
Couldnt load file on VM
loading craig_A_matrix.csv from hard drive...
craig_A_matrix.csv loaded successfuly
Couldnt load file on VM
loading craig_L_matrix.csv from hard drive...
craig_L_matrix.csv loaded successfuly
Couldnt load file on VM
loading craig_S_norm_env_flow_matrix.csv from hard drive...
craig_S_norm_env_flow_matrix.csv loaded successfuly
Couldnt load file on VM
loading craig_C_characterization_matrix.csv from hard drive...
craig_C_characterization_matrix.csv loaded successfuly
Table CS calculated successfuly

 All tables loaded successfuly


# Example for calculation steps

Eextract emission coeficients
1. creaet a Y vector of all 1s
2. multiply CSLYdiag

In [22]:
#1. create a Y vector = 
tables_Y["ones"] = 1
#y = tables_Y.["ones"=1]
tables_Y.shape
y = pd.DataFrame(tables_Y['ones'])


ones
CA-AB Canola (including rapeseed)                           1
      Oilseeds (except canola)                              1
      Wheat                                                 1
      Grains (except wheat)                                 1
      Fresh potatoes                                        1
...                                                       ...
WM    Membership organisation services n.e.c. (91)          1
      Recreational, cultural and sporting services (92)     1
      Other services (93)                                   1
      Private households with employed persons (95)         1
      Extra-territorial organizations and bodies            1

[16196 rows x 1 columns]

In [26]:

#2. multiply CSLy
final_demand_category = tables_Y["ones"]


#This line findes the final demand category to analyze (province, sector, sub sector)
on_final_res_structures = y
# transformation to df requiered for dot() multiplication
on_final_res_structures = pd.DataFrame(on_final_res_structures)
on_final_res_structures

# 1. this line calc total output of sector. X = (L)16Kx16K * (Y)16Kx1
on_res_struc_X = vector_X(tables_L, on_final_res_structures)
#call diagonolised X table. [0]  is the vector_X series, [1] is the diagonolized df

#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
CO2e = env_impact(tables_CS, on_res_struc_X[1])

#3. assign title to column  
CO2e = pd.DataFrame(CO2e).rename(
    columns = {'Climate change, short term': 'short term coefficient'})
CO2e

,,short term coefficient
,,kg CO2 eq (short)
MX,Gasoline Type Jet Fuel,6.400072e+07
NL,"Electricity by tide, wave, ocean",3.194844e+07
PT,Lignite/Brown Coal,2.237895e+07
NL,Electricity by solar thermal,6.939210e+06
FI,Gasoline Type Jet Fuel,4.857087e+06
...,...,...
IN,Extra-territorial organizations and bodies,0.000000e+00
CA-BC,"Cigarettes, cigars, chewing and smoking tobacco",0.000000e+00
BE,Gas Coke,0.000000e+00


In [ ]:
CO2e.to_csv(expath + tests + 'CA_CO2e_commodity_coeficients.csv')

#### 1.defind final demand vector Y

In [27]:
#define province /sector /sub sector
province = 'CA-ON'
sector = 'Gross fixed capital formation, Construction'
sub_sector = 'Residential structures'
# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector, sub_sector)
final_demand_category

#This line findes the final demand category to analyze (province, sector, sub sector)
on_final_res_structures = tables_Y[(final_demand_category)]
# transformation to df requiered for dot() multiplication
on_final_res_structures = pd.DataFrame(on_final_res_structures)
on_final_res_structures

CA-ON
                                                        Gross fixed capital formation, Construction
                                                                             Residential structures
CA-AB Canola (including rapeseed)                                                               0.0
      Oilseeds (except canola)                                                                  0.0
      Wheat                                                                                     0.0
      Grains (except wheat)                                                                     0.0
      Fresh potatoes                                                                            0.0
...                                                                                             ...
WM    Membership organisation services n.e.c. (91)                                              0.0
      Recreational, cultural and sporting services (92)                                         0.0
      Other services (93)                                                                       0.0
      Private households with employed persons (95)                                             0.0
      Extra-territorial organizations and bodies                                                0.0

[16196 rows x 1 columns]

#### 2.Find total outputX and diagonolize X= [(L)(Y)]


In [28]:
# 1. this line calc total output of sector. X = (L)16Kx16K * (Y)16Kx1
on_res_struc_X = vector_X(tables_L, on_final_res_structures)
#call diagonolised X table. [0]  is the vector_X series, [1] is the diagonolized df


#### 3.Calc environmental impact

In [29]:
#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
on_res_struct_CO2e = env_impact(tables_CS, on_res_struc_X[1])

In [30]:
#3. assign title to column  
on_res_struct_CO2e = pd.DataFrame(on_res_struct_CO2e).rename(
    columns = {'Climate change, short term': final_demand_category})
on_res_struct_CO2e

(CA-ON, Gross fixed capital formation, Construction, Residential structures)
                                                                                                                    kg CO2 eq (short)
CA-ON Residential buildings                                                                    8.742871e+08                          
      Ready-mixed concrete                                                                     7.027790e+08                          
CN    Electricity by coal                                                                      4.850241e+08                          
CA-ON Real estate brokerage and other services relate...                                       2.840990e+08                          
      Road transportation services for specialized fr...                                       2.714637e+08                          
...                                                                                                     ...                          
IE    Patent Fuel                                                                              0.000000e+00                          
      Other Bituminous Coal                                                                    0.000000e+00                          
      Coking Coal                                                                              0.000000e+00                          
      Manure (biogas treatment)                                                                0.000000e+00                          
WM    Extra-territorial organizations and bodies                                               0.000000e+00                          

[16196 rows x 1 columns]

# Canada green GFCFC


In [3]:
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
province = 'CA-ON'
sector = 'Gross fixed capital formation, Construction'
sub_sector = 'Residential structures'
# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector,sub_sector)
final_demand_category

#This line findes the final demand category to analyze (province, sector, sub sector)
p_GFCFC_Y  = tables_Y[(final_demand_category)]

Y_diag = pd.DataFrame(np.diag(p_GFCFC_Y),
                                    index=p_GFCFC_Y.index, columns=p_GFCFC_Y.index)
Y_diag

CA-AB  \
                                                        Canola (including rapeseed)   
CA-AB Canola (including rapeseed)                                               0.0   
      Oilseeds (except canola)                                                  0.0   
      Wheat                                                                     0.0   
      Grains (except wheat)                                                     0.0   
      Fresh potatoes                                                            0.0   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                              0.0   
      Recreational, cultural and sporting services (92)                         0.0   
      Other services (93)                                                       0.0   
      Private households with employed persons (95)                             0.0   
      Extra-territorial organizations and bodies                                0.0   

                                                                                  \
                                                        Oilseeds (except canola)   
CA-AB Canola (including rapeseed)                                            0.0   
      Oilseeds (except canola)                                               0.0   
      Wheat                                                                  0.0   
      Grains (except wheat)                                                  0.0   
      Fresh potatoes                                                         0.0   
...                                                                          ...   
WM    Membership organisation services n.e.c. (91)                           0.0   
      Recreational, cultural and sporting services (92)                      0.0   
      Other services (93)                                                    0.0   
      Private households with employed persons (95)                          0.0   
      Extra-territorial organizations and bodies                             0.0   

                                                               \
                                                        Wheat   
CA-AB Canola (including rapeseed)                         0.0   
      Oilseeds (except canola)                            0.0   
      Wheat                                               0.0   
      Grains (except wheat)                               0.0   
      Fresh potatoes                                      0.0   
...                                                       ...   
WM    Membership organisation services n.e.c. (91)        0.0   
      Recreational, cultural and sporting services (92)   0.0   
      Other services (93)                                 0.0   
      Private households with employed persons (95)       0.0   
      Extra-territorial organizations and bodies          0.0   

                                                                               \
                                                        Grains (except wheat)   
CA-AB Canola (including rapeseed)                                         0.0   
      Oilseeds (except canola)                                            0.0   
      Wheat                                                               0.0   
      Grains (except wheat)                                               0.0   
      Fresh potatoes                                                      0.0   
...                                                                       ...   
WM    Membership organisation services n.e.c. (91)                        0.0   
      Recreational, cultural and sporting services (92)                   0.0   
      Other services (93)                                                 0.0   
      Private households with employed persons (95)                       0.0   
      Extra-territorial organizatio

In [8]:
# canada green GFCFC

for p in provinces:
    # 1. calculate first province to create DF stucture
    #This line findes the final demand category to analyze (province, sector, sub sector)
    p_GFCFC_Y  = tables_Y[(p, sector)].sum(axis=1)
    print (p +' selected ')
    Y_diag = pd.DataFrame(np.diag(p_GFCFC_Y),
                                        index=p_GFCFC_Y.index, columns=p_GFCFC_Y.index)
    Y_diag
    ## This line calculates the total output enmissions
    df_LY = tables_L.dot(Y_diag)
    print (p +' df_LY calculated ')
    Y_diag = pd.DataFrame(np.diag(p_GFCFC_Y),
                                        index=p_GFCFC_Y.index, columns=p_GFCFC_Y.index)
    Y_diag
    ## This line calculates the total output enmissions
    df_LY = tables_L.dot(Y_diag)
    print ('in '+p+', df_LY was diagonolized successfuly')
    # Collapse LY
    df_LY = df_LY.sum(1)
    # diagnolize LY
    LY_diag = pd.DataFrame(np.diag(df_LY),index=df_LY.index, columns= df_LY.index)
    LY_diag

    total_output = tables_C.dot(tables_S).dot(LY_diag)
    total_output
    print( p+' residential structure - Green, calculated successfuly')
    report = total_output.loc[('Climate change, short term','kg CO2 eq (short)')].T#.sort_values(ascending=False)
    p_gfgfc_res_green = pd.DataFrame(report)
    #rename_column
    p_gfgfc_res_green.rename(columns={'Climate change, short term' : p}).droplevel(1, axis=1)
    print( p+' residential structure - Green, calculated duccessfuly')
    p_gfgfc_res_green.to_csv(expath+ tests + p+'_gfgfc_green.csv')

CA-NL selected 
CA-NL df_LY calculated 
in CA-NL, df_LY was diagonolized successfuly
CA-NL residential structure - Green, calculated successfuly
CA-NL residential structure - Green, calculated duccessfuly
CA-PE selected 
CA-PE df_LY calculated 
in CA-PE, df_LY was diagonolized successfuly
CA-PE residential structure - Green, calculated successfuly
CA-PE residential structure - Green, calculated duccessfuly
CA-NS selected 
CA-NS df_LY calculated 
in CA-NS, df_LY was diagonolized successfuly
CA-NS residential structure - Green, calculated successfuly
CA-NS residential structure - Green, calculated duccessfuly
CA-NB selected 
CA-NB df_LY calculated 
in CA-NB, df_LY was diagonolized successfuly
CA-NB residential structure - Green, calculated successfuly
CA-NB residential structure - Green, calculated duccessfuly
CA-QC selected 
CA-QC df_LY calculated 
in CA-QC, df_LY was diagonolized successfuly
CA-QC residential structure - Green, calculated successfuly
CA-QC residential structure - Green

## Create a CA GFFCF Constuction DF (for Sankey from files (green))

In [2]:
prov_df = pd.DataFrame()
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
data_path  ='C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/CA_gfcfc_green/'

# Create index columns
df_index = df = pd.read_csv(data_path + 'CA-ON_gfgfc_green.csv', ).iloc[:,:2]#header=[0], index_col=[0,1])
df_index  =df_index[1:].rename(columns = {'Unnamed: 0':'country','Unnamed: 1':'commodity'})
df_index

# Loop all provinces into one emission df
for p in provinces:
    df = pd.read_csv(data_path + p+'_gfgfc_green.csv', )#header=[0], index_col=[0,1])
    #clean df and rename colums
    df = df[1:].rename(columns = {'Unnamed: 0':'country','Unnamed: 1':'commodity', 'Climate change, short term':p})
    #CA_green_list.append(df)

    # Set the first two columns as the index
    #df.set_index(df.columns[:2], inplace=True)

    # Add the third column to the new_df
    prov_df = pd.concat([prov_df, df.iloc[:, 2]], axis=1, sort=False)

prov_df


# join DFs


df1_g = df_index.join(prov_df)
df1_g

,country,commodity,CA-NL,CA-PE,CA-NS,CA-NB,CA-QC,CA-ON,CA-MB,CA-SK,CA-AB,CA-BC,CA-YT,CA-NT,CA-NU
1,CA-AB,Canola (including rapeseed),37957.56163587899,7918.865130840526,51182.06039669682,59136.570578897496,799786.9930121942,1356317.5842970663,146387.47652393652,279988.6411302445,2803808.4274619445,1348128.505087524,11904.320417839364,23921.391450365958,19423.861586562063
2,CA-AB,Oilseeds (except canola),610.7757666335799,105.93685196205891,681.1443255827835,682.5451321235437,8631.46799662124,17494.324075566954,2691.6514942746044,5156.771258141221,57040.370093484446,16001.248290497915,226.54702576907155,373.2617313789679,278.2152096819848
3,CA-AB,Wheat,39539.066328103676,4191.233271388133,28525.743727946894,24408.483841769965,564592.7262281325,615462.4183617553,97668.6414392006,285797.0563729154,1597810.9884603259,725720.1409988746,6473.984402962266,11875.969797615775,10319.3293557608
4,CA-AB,Grains (except wheat),16174.130031893223,2040.702881430062,13004.147022690366,11691.383489847058,139868.25170579934,278649.4730289555,65376.70171102853,96624.47878461279,1010775.7864755021,573378.1206865136,4681.40941836262,6796.939966650249,6567.180223727508
5,CA-AB,Fresh potatoes,4555.109947436937,649.1412921852169,3932.0651761434597,3272.8017785311617,50477.15937531556,172495.973349114,14967.491671400063,30039.980628524943,381674.25463725993,71817.53901456832,1129.0086443518194,1821.1072350395618,1970.2209252681228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16192,WM,Membership organisation services n.e.c. (91),242.84911206269638,27.758344738980515,205.77370597527545,159.82064057746356,1409.4222819276347,3127.4193702274365,394.5901842307075,504.01378000770075,2174.1187027096735,1633.169520893644,26.925565097995154,40.597401216676005,68.62977459663517
16193,WM,"Recreational, cultural and sporting services (92)",1901.2793447467031,253.40357120876362,1634.0254675919891,1319.2715511799413,13395.93891275388,32810.95563397812,3477.060485044726,4698.5281028232375,20222.232290100896,15039.452031036646,226.7291685382643,389.62507284769043,628.32650591116
16194,WM,Other services (93),3066.1064924108664,354.689036941735,2590.472665379442,2003.485207512703,17362.91573367478,39913.765884435015,4952.177231885136,6237.305354751799,26704.14635373761,20641.425982989793,348.2460043003641,534.9997934125331,903.1328529306223
16195,WM,Private households with employed persons (95),171.85754040097237,20.011389305335346,146.98548288745374,108.99351937176381,1128.6381470420329,2519.704918671701,303.17500850678636,388.04289325452396,1579.3086023012208,1248.1623621481904,20.959097285474908,27.899727970534315,53.74407087088918


In [65]:
df1_g.to_csv(expath + tests + 'CA_gfcfc_green.csv')

# GFCF Construction. Ontario Calc

In [8]:
# this calculates the env impact for ()'CA-ON', 'Gross fixed capital formation, Construction')
#define province /sector /sub sector
province = 'CA-ON'
sector = 'Gross fixed capital formation, Construction'
sub_sector = 'Residential structures'
# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector)
final_demand_category

#This line findes the final demand category to analyze (province, sector, sub sector)
on_GFCFC_Y  = tables_Y[(final_demand_category)]
# transformation to df requiered for dot() multiplication
on_GFCFC_Y  = pd.DataFrame(on_GFCFC_Y)
on_GFCFC_Y 

# 1. this line calc total output of sector. X = (L)16Kx16K * (Y)16Kx1
on_GFCFC_X = vector_X(tables_L, on_GFCFC_Y)
#call diagonolised X table. [0]  is the vector_X series, [1] is the diagonolized df

#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
on_GFCFC_CO2e = env_impact(tables_CS, on_GFCFC_X[1])

#3. assign title to column  
on_GFCFC_CO2e = pd.DataFrame(on_GFCFC_CO2e).rename(
    columns = {'Climate change, short term': final_demand_category})
on_GFCFC_CO2e

(CA-ON, Gross fixed capital formation, Construction)
                                                                                            kg CO2 eq (short)
CA-ON Ready-mixed concrete                                                                     1.910424e+09  
CN    Electricity by coal                                                                      9.533534e+08  
CA-ON Residential buildings                                                                    8.965944e+08  
      Concrete products                                                                        7.327438e+08  
CN    Basic iron and steel and of ferro-alloys and fi...                                       5.020737e+08  
...                                                                                                     ...  
US    Secondary glass for treatment, Re-processing of...                                       0.000000e+00  
CA-BC Other metal ores and concentrates                                                        0.000000e+00  
CA-ON Trusteed pension fund services                                                           0.000000e+00  
      Mutual funds (cost of service) and other simila...                                       0.000000e+00  
WM    Extra-territorial organizations and bodies                                               0.000000e+00  

[16196 rows x 1 columns]

In [88]:
on_GFCFC_CO2e.to_csv(expath+exports+'on_GFCFC_CO2e.csv')

## Residential structure loop for canada

In [66]:
# define variables


province = 'CA-ON'
sector = 'Gross fixed capital formation, Construction'
sub_sector = 'Residential structures'
# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector,sub_sector)
final_demand_category

#This line findes the final demand category to analyze (province, sector, sub sector)
p_GFCFC_Y  = tables_Y[(final_demand_category)]

Y_diag = pd.DataFrame(np.diag(p_GFCFC_Y),
                                    index=p_GFCFC_Y.index, columns=p_GFCFC_Y.index)
Y_diag


CA-AB  \
                                                        Canola (including rapeseed)   
CA-AB Canola (including rapeseed)                                               0.0   
      Oilseeds (except canola)                                                  0.0   
      Wheat                                                                     0.0   
      Grains (except wheat)                                                     0.0   
      Fresh potatoes                                                            0.0   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                              0.0   
      Recreational, cultural and sporting services (92)                         0.0   
      Other services (93)                                                       0.0   
      Private households with employed persons (95)                             0.0   
      Extra-territorial organizations and bodies                                0.0   

                                                                                  \
                                                        Oilseeds (except canola)   
CA-AB Canola (including rapeseed)                                            0.0   
      Oilseeds (except canola)                                               0.0   
      Wheat                                                                  0.0   
      Grains (except wheat)                                                  0.0   
      Fresh potatoes                                                         0.0   
...                                                                          ...   
WM    Membership organisation services n.e.c. (91)                           0.0   
      Recreational, cultural and sporting services (92)                      0.0   
      Other services (93)                                                    0.0   
      Private households with employed persons (95)                          0.0   
      Extra-territorial organizations and bodies                             0.0   

                                                               \
                                                        Wheat   
CA-AB Canola (including rapeseed)                         0.0   
      Oilseeds (except canola)                            0.0   
      Wheat                                               0.0   
      Grains (except wheat)                               0.0   
      Fresh potatoes                                      0.0   
...                                                       ...   
WM    Membership organisation services n.e.c. (91)        0.0   
      Recreational, cultural and sporting services (92)   0.0   
      Other services (93)                                 0.0   
      Private households with employed persons (95)       0.0   
      Extra-territorial organizations and bodies          0.0   

                                                                               \
                                                        Grains (except wheat)   
CA-AB Canola (including rapeseed)                                         0.0   
      Oilseeds (except canola)                                            0.0   
      Wheat                                                               0.0   
      Grains (except wheat)                                               0.0   
      Fresh potatoes                                                      0.0   
...                                                                       ...   
WM    Membership organisation services n.e.c. (91)                        0.0   
      Recreational, cultural and sporting services (92)                   0.0   
      Other services (93)                                                 0.0   
      Private households with employed persons (95)                       0.0   
      Extra-territorial organizatio

In [67]:
## This line calculates the total output enmissions
L = tables_L.dot(Y_diag)
L


CA-AB  \
                                                        Canola (including rapeseed)   
CA-AB Canola (including rapeseed)                                               0.0   
      Oilseeds (except canola)                                                  0.0   
      Wheat                                                                     0.0   
      Grains (except wheat)                                                     0.0   
      Fresh potatoes                                                            0.0   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                              0.0   
      Recreational, cultural and sporting services (92)                         0.0   
      Other services (93)                                                       0.0   
      Private households with employed persons (95)                             0.0   
      Extra-territorial organizations and bodies                                0.0   

                                                                                  \
                                                        Oilseeds (except canola)   
CA-AB Canola (including rapeseed)                                            0.0   
      Oilseeds (except canola)                                               0.0   
      Wheat                                                                  0.0   
      Grains (except wheat)                                                  0.0   
      Fresh potatoes                                                         0.0   
...                                                                          ...   
WM    Membership organisation services n.e.c. (91)                           0.0   
      Recreational, cultural and sporting services (92)                      0.0   
      Other services (93)                                                    0.0   
      Private households with employed persons (95)                          0.0   
      Extra-territorial organizations and bodies                             0.0   

                                                               \
                                                        Wheat   
CA-AB Canola (including rapeseed)                         0.0   
      Oilseeds (except canola)                            0.0   
      Wheat                                               0.0   
      Grains (except wheat)                               0.0   
      Fresh potatoes                                      0.0   
...                                                       ...   
WM    Membership organisation services n.e.c. (91)        0.0   
      Recreational, cultural and sporting services (92)   0.0   
      Other services (93)                                 0.0   
      Private households with employed persons (95)       0.0   
      Extra-territorial organizations and bodies          0.0   

                                                                               \
                                                        Grains (except wheat)   
CA-AB Canola (including rapeseed)                                         0.0   
      Oilseeds (except canola)                                            0.0   
      Wheat                                                               0.0   
      Grains (except wheat)                                               0.0   
      Fresh potatoes                                                      0.0   
...                                                                       ...   
WM    Membership organisation services n.e.c. (91)                        0.0   
      Recreational, cultural and sporting services (92)                   0.0   
      Other services (93)                                                 0.0   
      Private households with employed persons (95)                       0.0   
      Extra-territorial organizatio

In [70]:
# Collapse LY
LY = L.sum(1)
# diagnolize LY
LY_diag = pd.DataFrame(np.diag(LY),
                        index=LY.index, columns=LY.index)
LY_diag

CA-AB  \
                                                        Canola (including rapeseed)   
CA-AB Canola (including rapeseed)                                      1.068137e+06   
      Oilseeds (except canola)                                         0.000000e+00   
      Wheat                                                            0.000000e+00   
      Grains (except wheat)                                            0.000000e+00   
      Fresh potatoes                                                   0.000000e+00   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                     0.000000e+00   
      Recreational, cultural and sporting services (92)                0.000000e+00   
      Other services (93)                                              0.000000e+00   
      Private households with employed persons (95)                    0.000000e+00   
      Extra-territorial organizations and bodies                       0.000000e+00   

                                                                                  \
                                                        Oilseeds (except canola)   
CA-AB Canola (including rapeseed)                                       0.000000   
      Oilseeds (except canola)                                      14487.378829   
      Wheat                                                             0.000000   
      Grains (except wheat)                                             0.000000   
      Fresh potatoes                                                    0.000000   
...                                                                          ...   
WM    Membership organisation services n.e.c. (91)                      0.000000   
      Recreational, cultural and sporting services (92)                 0.000000   
      Other services (93)                                               0.000000   
      Private households with employed persons (95)                     0.000000   
      Extra-territorial organizations and bodies                        0.000000   

                                                                        \
                                                                 Wheat   
CA-AB Canola (including rapeseed)                             0.000000   
      Oilseeds (except canola)                                0.000000   
      Wheat                                              471338.638137   
      Grains (except wheat)                                   0.000000   
      Fresh potatoes                                          0.000000   
...                                                                ...   
WM    Membership organisation services n.e.c. (91)            0.000000   
      Recreational, cultural and sporting services (92)       0.000000   
      Other services (93)                                     0.000000   
      Private households with employed persons (95)           0.000000   
      Extra-territorial organizations and bodies              0.000000   

                                                                               \
                                                        Grains (except wheat)   
CA-AB Canola (including rapeseed)                                     0.00000   
      Oilseeds (except canola)                                        0.00000   
      Wheat                                                           0.00000   
      Grains (except wheat)                                      213405.35227   
      Fresh potatoes                                                  0.00000   
...                                                                       ...   
WM    Membership organisation services n.e.c. (91)                    0.00000   
      Recreational, cultural and sporting services (92)               0.00000   
      Other services (93)                                             0.00000   

In [71]:
total_output = tables_C.dot(tables_S).dot(LY_diag)
total_output

CA-AB  \
                                                                Canola (including rapeseed)   
Impact category                               CF unit                                         
Climate change, ecosystem quality, long term  PDF.m2.yr                       343778.300384   
Climate change, ecosystem quality, short term PDF.m2.yr                       152745.811204   
Climate change, human health, long term       DALY                                 1.561815   
Climate change, human health, short term      DALY                                 0.707237   
Climate change, long term                     kg CO2 eq (long)                857506.436277   
Climate change, short term                    kg CO2 eq (short)               864186.365798   
Freshwater acidification                      PDF.m2.yr                            0.000000   
                                              kg SO2 eq                            0.000000   
Freshwater ecotoxicity                        CTUe                                 0.000000   
Freshwater ecotoxicity, long term             PDF.m2.yr                            0.000000   
Freshwater ecotoxicity, short term            PDF.m2.yr                            0.000000   
Freshwater eutrophication                     PDF.m2.yr                            0.000000   
                                              kg PO4 P-lim eq                      0.000000   
Human toxicity cancer                         CTUh                                 0.000000   
Human toxicity cancer, long term              DALY                                 0.000000   
Human toxicity cancer, short term             DALY                                 0.000000   
Human toxicity non cancer                     CTUh                                 0.000000   
Human toxicity non-cancer, long term          DALY                                 0.000000   
Human toxicity non-cancer, short term         DALY                                 0.000000   
Marine acidification, long term               PDF.m2.yr                        64270.589815   
Marine acidification, short term              PDF.m2.yr                         6976.741658   
Marine eutrophication                         PDF.m2.yr                            0.000000   
                                              kg N N-lim eq                        0.000000   
Ozone Layer Depletion                         kg  CFC-11 eq                        0.000000   
Ozone layer depletion                         DALY                                 0.000000   
Particulate matter formation                  DALY                                 0.000000   
                                              kg PM2.5 eq                          0.000000   
Photochemical oxidant formation               DALY                                 0.000000   
                                              kg NMVOC eq                          2.294367   
Terrestrial acidification                     PDF.m2.yr                            0.000000   
                                              kg SO2 eq                            0.000000   
Water use                                     m3                               49181.252018   
Energy use                                    TJ                                   4.456919   

                                                                                          \
                                                                Oilseeds (except canola)   
Impact category                               CF unit                                      
Climate change, ecosystem quality, long term  PDF.m2.yr                     4.445061e+03   
Climate change, ecosystem quality, short term PDF.m2.yr                     1.968108e+03   
Climate change, human health, long term       DALY                          2.019477e-02   
Climate change, human health, short term      DALY                          9.112545e-03   
Climate change, long 

In [84]:
report = total_output.loc[('Climate change, short term','kg CO2 eq (short)')].T#.sort_values(ascending=False)
prov_gfgfc_res_green = pd.DataFrame(report)
#rename_column
prov_gfgfc_res_green.rename(columns={'Climate change, short term' : province}).droplevel(1, axis=1)


CA-ON
CA-AB Canola (including rapeseed)                        864186.365798
      Oilseeds (except canola)                            11134.824688
      Wheat                                              381414.094931
      Grains (except wheat)                              173814.712483
      Fresh potatoes                                     114681.488089
...                                                                ...
WM    Membership organisation services n.e.c. (91)         1638.339680
      Recreational, cultural and sporting services (92)   18566.098153
      Other services (93)                                 20666.538245
      Private households with employed persons (95)        1395.644907
      Extra-territorial organizations and bodies              0.000000

[16196 rows x 1 columns]

In [72]:
report = total_output.loc[('Climate change, short term','kg CO2 eq (short)')].T.sort_values(ascending=False)
prov_gfgfc_res_green = pd.DataFrame(report)
#rename_column
prov_gfgfc_res_green.rename(columns={'Climate change, short term' : province}).droplevel(1, axis=1)


CA-ON
CA-ON Residential buildings                               8.742871e+08
      Ready-mixed concrete                                7.027790e+08
CN    Electricity by coal                                 4.850241e+08
CA-ON Real estate brokerage and other services relate...  2.840990e+08
      Road transportation services for specialized fr...  2.714637e+08
...                                                                ...
IE    Patent Fuel                                         0.000000e+00
      Other Bituminous Coal                               0.000000e+00
      Coking Coal                                         0.000000e+00
      Manure (biogas treatment)                           0.000000e+00
WM    Extra-territorial organizations and bodies          0.000000e+00

[16196 rows x 1 columns]

In [46]:
prov_gfgfc_res_green.to_csv(expath+ tests +'prov_gfgfc_res_green.csv')

calculate gfcf construction residential structures - loop all provinces

loop for all provinces

In [8]:
# define variables

provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
sector = 'Gross fixed capital formation, Construction'
sub_sector = 'Residential structures'


for p in provinces:
    # 1. calculate first province to create DF stucture
    #This line findes the final demand category to analyze (province, sector, sub sector)
    p_GFCFC_Y  = tables_Y[(p, sector,sub_sector)]
    print (p +' selected ')
    Y_diag = pd.DataFrame(np.diag(p_GFCFC_Y),
                                        index=p_GFCFC_Y.index, columns=p_GFCFC_Y.index)
    Y_diag
    ## This line calculates the total output enmissions
    df_LY = tables_L.dot(Y_diag)
    print (p +' df_LY calculated ')
    Y_diag = pd.DataFrame(np.diag(p_GFCFC_Y),
                                        index=p_GFCFC_Y.index, columns=p_GFCFC_Y.index)
    Y_diag
    ## This line calculates the total output enmissions
    df_LY = tables_L.dot(Y_diag)
    print ('in '+p+', df_LY was diagonolized successfuly')
    # Collapse LY
    df_LY = df_LY.sum(1)
    # diagnolize LY
    LY_diag = pd.DataFrame(np.diag(df_LY),index=df_LY.index, columns= df_LY.index)
    LY_diag

    total_output = tables_C.dot(tables_S).dot(LY_diag)
    total_output
    print( p+' residential structure - Green, calculated successfuly')
    report = total_output.loc[('Climate change, short term','kg CO2 eq (short)')].T#.sort_values(ascending=False)
    p_gfgfc_res_green = pd.DataFrame(report)
    #rename_column
    p_gfgfc_res_green.rename(columns={'Climate change, short term' : p}).droplevel(1, axis=1)
    print( p+' residential structure - Green, calculated duccessfuly')
    p_gfgfc_res_green.to_csv(expath+ tests + p+'_gfgfc_res_green.csv')


CA-NL selected 
CA-NL df_LY calculated 
in CA-NL, df_LY was diagonolized successfuly
CA-NL residential structure - Green, calculated duccessfuly
CA-NL residential structure - Green, calculated duccessfuly
CA-PE selected 
CA-PE df_LY calculated 
in CA-PE, df_LY was diagonolized successfuly
CA-PE residential structure - Green, calculated duccessfuly
CA-PE residential structure - Green, calculated duccessfuly
CA-NS selected 
CA-NS df_LY calculated 
in CA-NS, df_LY was diagonolized successfuly
CA-NS residential structure - Green, calculated duccessfuly
CA-NS residential structure - Green, calculated duccessfuly
CA-NB selected 
CA-NB df_LY calculated 
in CA-NB, df_LY was diagonolized successfuly
CA-NB residential structure - Green, calculated duccessfuly
CA-NB residential structure - Green, calculated duccessfuly
CA-QC selected 
CA-QC df_LY calculated 
in CA-QC, df_LY was diagonolized successfuly
CA-QC residential structure - Green, calculated duccessfuly
CA-QC residential structure - Green

In [5]:
p = provinces[0]
print ('in '+p+', df_LY was diagonolized successfuly')


in CA-NL, df_LY was diagonolized successfuly


In [ ]:
LY_diag = pd.DataFrame(np.diag(LY),index=LY.index, columns=LY.index)
LY_diag
total_output = tables_C.dot(tables_S).dot(LY_diag)
total_output

In [ ]:

    Y_diag = pd.DataFrame(np.diag(p_GFCFC_Y),
                                        index=p_GFCFC_Y.index, columns=p_GFCFC_Y.index)
    Y_diag
    ## This line calculates the total output enmissions
    df_LY = tables_L.dot(Y_diag)
    # Collapse LY
    df_LY = df_LY.sum(1)
    # diagnolize LY
    LY_diag = pd.DataFrame(np.diag(df_LY),index=df_LY.index, columns= df_LY.index)
    LY_diag
    total_output = tables_C.dot(tables_S).dot(LY_diag)
    total_output
    print( p+' residential structure - Green, calculated duccessfuly')
    report = total_output.loc[('Climate change, short term','kg CO2 eq (short)')].T#.sort_values(ascending=False)
    p_gfgfc_res_green = pd.DataFrame(report)
    #rename_column
    p_gfgfc_res_green.rename(columns={'Climate change, short term' : province}).droplevel(1, axis=1)
    print( p+' residential structure - Green, calculated duccessfuly')
    prov_gfgfc_res_green.to_csv(expath+ tests + p+'_gfgfc_res_green.csv')

In [102]:
prov_gfgfc_res_green = pd.DataFrame() 

for  p in provinces[:1]:
    #This line findes the final demand category to analyze (province, sector, sub sector)
    p_GFCFC_Y  = tables_Y[(p, sector,sub_sector)]

    Y_diag = pd.DataFrame(np.diag(p_GFCFC_Y),
                                        index=p_GFCFC_Y.index, columns=p_GFCFC_Y.index)
    Y_diag
    ## This line calculates the total output enmissions
    df_LY = tables_L.dot(Y_diag)
    # Collapse LY
    df_LY = df_LY.sum(1)
    # diagnolize LY
    LY_diag = pd.DataFrame(np.diag(LY),
                        index=LY.index, columns=LY.index)
    LY_diag
    total_output = tables_C.dot(tables_S).dot(LY_diag)
    total_output
    report = total_output.loc[('Climate change, short term','kg CO2 eq (short)')].T#.sort_values(ascending=False)
    p_gfgfc_res_green = pd.DataFrame(report)
    #rename_column
    p_gfgfc_res_green.rename(columns={'Climate change, short term' : province}).droplevel(1, axis=1)
    print(p+' was successfuly calculated.')
    # append df to list
    prov_gfgfc_res_green =pd.concat(p_gfgfc_res_green)
    print(p+' was added to list .')



TypeError: unhashable type: 'list'

In [ ]:
prov_gfgfc_res_green


In [17]:
# transformation to df requiered for dot() multiplication
p_GFCFC_Y  = pd.DataFrame(p_GFCFC_Y)
p_GFCFC_Y 


(CA-ON, Gross fixed capital formation, Construction, Residential structures)
                                                                                                                    kg CO2 eq (short)
CA-ON Residential buildings                                                                    8.742871e+08                          
      Ready-mixed concrete                                                                     7.027790e+08                          
CN    Electricity by coal                                                                      4.850241e+08                          
CA-ON Real estate brokerage and other services relate...                                       2.840990e+08                          
      Road transportation services for specialized fr...                                       2.714637e+08                          
...                                                                                                     ...                          
IE    Patent Fuel                                                                              0.000000e+00                          
      Other Bituminous Coal                                                                    0.000000e+00                          
      Coking Coal                                                                              0.000000e+00                          
      Manure (biogas treatment)                                                                0.000000e+00                          
WM    Extra-territorial organizations and bodies                                               0.000000e+00                          

[16196 rows x 1 columns]

In [19]:
tables_C

('CA-AB', '1,1,2,2-Tetrachloroethane', 'Air')  \
Impact category                               CF unit                                                            
Climate change, ecosystem quality, long term  PDF.m2.yr                                               0.000000   
Climate change, ecosystem quality, short term PDF.m2.yr                                               0.000000   
Climate change, human health, long term       DALY                                                    0.000000   
Climate change, human health, short term      DALY                                                    0.000000   
Climate change, long term                     kg CO2 eq (long)                                        0.000000   
Climate change, short term                    kg CO2 eq (short)                                       0.000000   
Freshwater acidification                      PDF.m2.yr                                               0.000000   
                                              kg SO2 eq                                               0.000000   
Freshwater ecotoxicity                        CTUe                                                    4.680000   
Freshwater ecotoxicity, long term             PDF.m2.yr                                               0.000000   
Freshwater ecotoxicity, short term            PDF.m2.yr                                               0.002564   
Freshwater eutrophication                     PDF.m2.yr                                               0.000000   
                                              kg PO4 P-lim eq                                         0.000000   
Human toxicity cancer                         CTUh                                                    0.000001   
Human toxicity cancer, long term              DALY                                                    0.000000   
Human toxicity cancer, short term             DALY                                                    0.000013   
Human toxicity non cancer                     CTUh                                                    0.000000   
Human toxicity non-cancer, long term          DALY                                                    0.000000   
Human toxicity non-cancer, short term         DALY                                                    0.000000   
Marine acidification, long term               PDF.m2.yr                                               0.000000   
Marine acidification, short term              PDF.m2.yr                                               0.000000   
Marine eutrophication                         PDF.m2.yr                                               0.000000   
                                              kg N N-lim eq                                           0.000000   
Ozone Layer Depletion                         kg  CFC-11 eq                                           0.000000   
Ozone layer depletion                         DALY                                                    0.000000   
Particulate matter formation                  DALY                                                    0.000000   
                                              kg PM2.5 eq                                             0.000000   
Photochemical oxidant formation               DALY                                                    0.000000   
                                              kg NMVOC eq                                             0.000000   
Terrestrial acidification                     PDF.m2.yr                                               0.000000   
                                              kg SO2 eq                                               0.000000   
Water use                                     m3                                                      0.000000   
Energy use                                    TJ                                                      0.000000   

                                                                 ('CA-AB',

In [6]:
tables_C = tables_C
p_x_res = total_co2e(p_GFCFC_Y, tables_C , tables_S , tables_L)
p_x_res

ValueError: No axis named 1 for object type Series

In [7]:

# 1. this line calc total output of sector. X = (L)16Kx16K * (Y)16Kx1
p_GFCFC_X = vector_X(tables_L, p_GFCFC_Y)
#call diagonolised X table. [0]  is the vector_X series, [1] is the diagonolized df

#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
p_GFCFC_CO2e = env_impact(tables_CS, p_GFCFC_X[1])

#3. assign title to column  
p_GFCFC_CO2e = pd.DataFrame(p_GFCFC_CO2e).rename(
    columns = {'Climate change, short term': final_demand_category})
p_GFCFC_CO2e

ValueError: No axis named 1 for object type Series

In [ ]:

#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
p_GFCFC_CO2e = env_impact(tables_CS, p_GFCFC_X[1])

#3. assign title to column  
p_GFCFC_CO2e = pd.DataFrame(p_GFCFC_CO2e).rename(
    columns = {'Climate change, short term': final_demand_category})
p_GFCFC_CO2e

#### GFCFC for Canada. loop

In [24]:
type(total_output)

pandas.core.series.Series

In [12]:
#1. this block calculates the GFCF, Construction emissions for every province in MTCO2e
#and appends it to a list. CIRAIG's method
gcf_construction_MTCO2ciraig = list()
for province in provinces:
    # 1. defince Y vector
    gfc_const = tables_Y[(province,'Gross fixed capital formation, Construction')]
    
    # 2. Calculating encironemtnal impact using CIRIAGS's IO tables
    total_output = tables_C.dot(tables_S).dot(tables_L).dot(gfc_const)
    total_output

    ## 3. This lin sorts out the short term impacts of the selected data
    report_list1 = total_output.loc[('Climate change, short term','kg CO2 eq (short)')].T.sort_values(ascending=False).sum()
    report_list1 = round(report_list1.sum()/1000000000, 3)
        # Calculation Message
    print(province +' GFC Construction '+ str(report_list1) +' MTCO2e')
    gcf_construction_MTCO2ciraig.append(report_list1)
print (gcf_construction_MTCO2ciraig)

CA-NL GFC Construction 1.925 MTCO2e
CA-PE GFC Construction 0.223 MTCO2e
CA-NS GFC Construction 1.776 MTCO2e
CA-NB GFC Construction 1.156 MTCO2e
CA-QC GFC Construction 13.709 MTCO2e
CA-ON GFC Construction 26.824 MTCO2e
CA-MB GFC Construction 3.124 MTCO2e
CA-SK GFC Construction 5.487 MTCO2e
CA-AB GFC Construction 20.052 MTCO2e
CA-BC GFC Construction 14.49 MTCO2e
CA-YT GFC Construction 0.195 MTCO2e
CA-NT GFC Construction 0.292 MTCO2e
CA-NU GFC Construction 0.364 MTCO2e
[1.925, 0.223, 1.776, 1.156, 13.709, 26.824, 3.124, 5.487, 20.052, 14.49, 0.195, 0.292, 0.364]


In [13]:
gcf_construction_MTCO2ciraig

[1.925,
 0.223,
 1.776,
 1.156,
 13.709,
 26.824,
 3.124,
 5.487,
 20.052,
 14.49,
 0.195,
 0.292,
 0.364]

# Dataframes

## All CA CFCFC loop by sup sector (green)

In [4]:
#list provinces and sectors to run in loop
sector = 'Gross fixed capital formation, Construction'
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
sub_sectors = tables_Y[('CA-ON', sector)]
sub_sec_list = sub_sectors.columns.values.tolist()
len(sub_sec_list) # list length is 54 sectors
#print (sub_sec_list)

['Accommodation and food services', 'Administrative and support, waste management and remediation services', 'Animal production', 'Arts, entertainment and recreation', 'Beverage and tobacco product manufacturing', 'Chemical manufacturing', 'Clothing and leather and allied product manufacturing', 'Computer and electronic product manufacturing', 'Construction', 'Crop production', 'Defence services', 'Educational services (private)', 'Educational services (public)', 'Electrical equipment, appliance and component manufacturing', 'Fabricated metal product manufacturing', 'Finance and insurance', 'Fishing, hunting and trapping', 'Food manufacturing', 'Forestry and logging', 'Furniture and related product manufacturing', 'Health care and social assistance', 'Holding companies', 'Hospitals', 'Information and cultural industries', 'Machinery manufacturing', 'Mining and quarrying (except oil and gas)', 'Miscellaneous manufacturing', 'Non-metallic mineral product manufacturing', 'Non-profit insti

In [5]:
# Designate final demand sector
# this demo uses ('CA-ON', 'Gross fixed capital formation, Construction')
#define province /sector /sub sector
province = provinces
sector = 'Gross fixed capital formation, Construction'
sub_sector = sub_sec_list


# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector, sub_sector)
final_demand_category

(['CA-NL',
  'CA-PE',
  'CA-NS',
  'CA-NB',
  'CA-QC',
  'CA-ON',
  'CA-MB',
  'CA-SK',
  'CA-AB',
  'CA-BC',
  'CA-YT',
  'CA-NT',
  'CA-NU'],
 'Gross fixed capital formation, Construction',
 ['Accommodation and food services',
  'Administrative and support, waste management and remediation services',
  'Animal production',
  'Arts, entertainment and recreation',
  'Beverage and tobacco product manufacturing',
  'Chemical manufacturing',
  'Clothing and leather and allied product manufacturing',
  'Computer and electronic product manufacturing',
  'Construction',
  'Crop production',
  'Defence services',
  'Educational services (private)',
  'Educational services (public)',
  'Electrical equipment, appliance and component manufacturing',
  'Fabricated metal product manufacturing',
  'Finance and insurance',
  'Fishing, hunting and trapping',
  'Food manufacturing',
  'Forestry and logging',
  'Furniture and related product manufacturing',
  'Health care and social assistance',
  'Hol

the loop

In [ ]:
# create lists to stor data
ca_master_gfcfc_df_green = pd.DataFrame()
province_gfcfc_list = []

for p in provinces[0:2]:
    for s in sub_sector[0:1]:
        final_demand_category = (p, sector, s)
        #print ('Caclc ' + str(final_demand_category))
        #print (tables_Y[(p, sector, s)].sum())
        #---- calculationcode goes here ----
         # isolate final demnad vector

        on_gfcf_sub_Y = tables_Y[(final_demand_category)]
        #transform series into df
        on_gfcf_sub_Y = pd.DataFrame(on_gfcf_sub_Y)
        on_gfcf_sub_Y
        print ('in ',p, ' claculating ', s)
        # 1.2 this line calc the total intermediate output of sector. D = (A)16Kx16K * (Y)16Kx1K
        on_gfcf_sub_LY= vector_X(tables_L, on_gfcf_sub_Y)
        #call diagonolised table. [0]  is the vector_X series, [1] is the diagonolized df

        #2. Calculate environmental impact. returnes kgCO2e for called total output vector  
        # call [1] for diagonal or [0] sector summary
        on_gfcf_sub_CSX = env_impact(tables_CS, on_gfcf_sub_LY[1])

        #3. assign title to column
        on_gfcf_sub_CSX = pd.DataFrame(on_gfcf_sub_CSD).rename(
            columns = {'Climate change, short term': (p+'_gfcfc_'+ s)})
        on_gfcf_sub_CSX

        
        #appending result to DF
        left = on_gfcfc_sub_materials
        right = on_gfcf_sub_CSX
        on_gfcfc_sub_materials = left.join(right, rsuffix = s)

        #------------------------------------
        print ('In ' + p+ ', '+s +'vector appended')
        on_gfcfc_sub_materials
    print ('---- '+p+ ' CALCULATED SUCCESSFULY ---')
    on_gfcfc_sub_materials.to_csv(expath+tests+ p +'gfcfc_sub_materials_green.csv')
    province_gfcfc_list = province_gfcfc_list.append(on_gfcfc_sub_materials)
    ca_master_gfcfc_df_green = ca_master_gfcfc_df_green.join(on_gfcfc_sub_materials, rsuffix = p)
    ca_master_gfcfc_df_green.to_csv(expath+tests+ 'CA_gfcfc_sub_materials_green.csv')
print ('--------------dataframe created sussessfuly-------')    

Residential building DF. CA by province. (CS)(LY)

In [17]:
#Define lists
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON',
              'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
# Create an empty Canada DF for joining loop calculations
ca_res_struct_CO2e = pd.DataFrame(index=tables_Y.index)

#This loop runs emission calculation on final demand category actoss all sectors
#define province /sector /sub sector
for p in provinces:
    province = p 
    sector = 'Gross fixed capital formation, Construction'
    sub_sector = 'Residential structures'
    # this parameter will propegate as a headar to the env impact table
    final_demand_category = (p, sector, sub_sector)
    print ('now cal ', p)

    #This line findes the final demand category to analyze (province, sector, sub sector)
    p_final_res_structures = tables_Y[(final_demand_category)]
    # transformation to df requiered for dot() multiplication
    p_final_res_structures = pd.DataFrame(p_final_res_structures)
    p_final_res_structures
    
    # 1. this line calc total output of sector. X = (L)16Kx16K * (Y)16Kx1
    p_res_struc_X = vector_X(tables_L, p_final_res_structures)
    #call diagonolised X table. [0]  is the vector_X series, [1] is the diagonolized df
    
    #2. Calculate environmental impact. returnes kgCO2e for called total output vector  
    # call [1] for diagonal or [0] sector summary
    p_res_struct_CO2e = env_impact(tables_CS, p_res_struc_X[1])

    #3. assign title to column
    p_res_struct_CO2e = pd.DataFrame(p_res_struct_CO2e).rename(
    columns = {'Climate change, short term': p})
    #frop column level
    p_res_struct_CO2e.droplevel([1], axis=1)
    print('emission calc for ', final_demand_category, 'completed')
    left = ca_res_struct_CO2e
    right = p_res_struct_CO2e
    ca_res_struct_CO2e =  left.join(right, rsuffix = p)
    
    print (p, 'CO2e vector appended')
print ('dataframe created successfuly')

now cal  CA-NL
emission calc for  ('CA-NL', 'Gross fixed capital formation, Construction', 'Residential structures') completed
CA-NL CO2e vector appended
now cal  CA-PE


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\472397338.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_res_struct_CO2e =  left.join(right, rsuffix = p)


emission calc for  ('CA-PE', 'Gross fixed capital formation, Construction', 'Residential structures') completed
CA-PE CO2e vector appended
now cal  CA-NS


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\472397338.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_res_struct_CO2e =  left.join(right, rsuffix = p)


emission calc for  ('CA-NS', 'Gross fixed capital formation, Construction', 'Residential structures') completed
CA-NS CO2e vector appended
now cal  CA-NB


...

C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\472397338.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_res_struct_CO2e =  left.join(right, rsuffix = p)


emission calc for  ('CA-NT', 'Gross fixed capital formation, Construction', 'Residential structures') completed
CA-NT CO2e vector appended
now cal  CA-NU


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\472397338.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_res_struct_CO2e =  left.join(right, rsuffix = p)


emission calc for  ('CA-NU', 'Gross fixed capital formation, Construction', 'Residential structures') completed
CA-NU CO2e vector appended
dataframe created successfuly


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\472397338.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_res_struct_CO2e =  left.join(right, rsuffix = p)


In [11]:
ca_res_struct_CO2e

(CA-NL, kg CO2 eq (short))  \
CA-AB Canola (including rapeseed)                                       8134.061117   
      Oilseeds (except canola)                                           116.812583   
      Wheat                                                             4782.514923   
      Grains (except wheat)                                             2146.384495   
      Fresh potatoes                                                     908.359628   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                        36.773496   
      Recreational, cultural and sporting services (92)                  319.007748   
      Other services (93)                                                446.467971   
      Private households with employed persons (95)                       30.144755   
      Extra-territorial organizations and bodies                           0.000000   

                                                         (CA-PE, kg CO2 eq (short))  \
CA-AB Canola (including rapeseed)                                       5298.022157   
      Oilseeds (except canola)                                            70.373384   
      Wheat                                                             2658.107335   
      Grains (except wheat)                                             1304.741933   
      Fresh potatoes                                                     434.816442   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                        14.042367   
      Recreational, cultural and sporting services (92)                  146.109091   
      Other services (93)                                                173.674950   
      Private households with employed persons (95)                       11.180163   
      Extra-territorial organizations and bodies                           0.000000   

                                                         (CA-NS, kg CO2 eq (short))  \
CA-AB Canola (including rapeseed)                                      34404.682726   
      Oilseeds (except canola)                                           446.114754   
      Wheat                                                            18533.176156   
      Grains (except wheat)                                             8335.597896   
      Fresh potatoes                                                    2556.150899   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                       106.133100   
      Recreational, cultural and sporting services (92)                  929.741450   
      Other services (93)                                               1287.356985   
      Private households with employed persons (95)                       84.402832   
      Extra-territorial organizations and bodies                           0.000000   

                                                         (CA-NB, kg CO2 eq (short))  \
CA-AB Canola (including rapeseed)                                      25269.332650   
      Oilseeds (except canola)                                           301.852831   
      Wheat                                                            10928.658722   
      Grains (except wheat)                                             4989.019757   
      Fresh potatoes                                                    1459.911952   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                        54.029848   
      Recreational, cultural and sporting services (92)                  490.528495   
      Other services (93)                                                640.171200   
      Private households with employed persons (95) 

In [130]:
ca_res_struct_CO2e.to_csv(expath + 'ca_res_struct_CO2e_green.csv')

GFCF DF. CA by province. (CS)(LY)

In [15]:
#Define lists
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON',
              'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
# Create an empty Canada DF for joining loop calculations
ca_gfcfc_CO2e = pd.DataFrame(index=tables_Y.index)

#This loop runs emission calculation on final demand category actoss all sectors
#define province /sector /sub sector
for p in provinces:
    province = p 
    sector = 'Gross fixed capital formation, Construction'
    sub_sector = 'Residential structures'
    # this parameter will propegate as a headar to the env impact table
    final_demand_category = (p, sector)
    print ('now cal ', p)

    #This line findes the final demand category to analyze (province, sector, sub sector)
    p_gfcfc = tables_Y[(final_demand_category)]
    # transformation to df requiered for dot() multiplication
    p_gfcfc = pd.DataFrame(p_gfcfc)
    p_gfcfc
    
    # 1. this line calc total output of sector. X = (L)16Kx16K * (Y)16Kx1
    p_gfcfc_X = vector_X(tables_L, p_gfcfc)
    #call diagonolised X table. [0]  is the vector_X series, [1] is the diagonolized df
    
    #2. Calculate environmental impact. returnes kgCO2e for called total output vector  
    # call [1] for diagonal or [0] sector summary
    p_gfcfc_CO2e = env_impact(tables_CS, p_gfcfc_X[1])

    #3. assign title to column
    p_gfcfc_CO2e = pd.DataFrame(p_gfcfc_CO2e).rename(
    columns = {'Climate change, short term': p})
    #frop column level
    p_gfcfc_CO2e.droplevel([1], axis=1)
    print('emission calc for ', final_demand_category, 'completed')
    left = ca_gfcfc_CO2e
    right = p_gfcfc_CO2e
    ca_gfcfc_CO2e =  left.join(right, rsuffix = p)
    
    print (p, 'CO2e vector appended')
print ('dataframe created successfuly')

now cal  CA-NL
emission calc for  ('CA-NL', 'Gross fixed capital formation, Construction') completed
CA-NL CO2e vector appended
now cal  CA-PE


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\1798386228.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_gfcfc_CO2e =  left.join(right, rsuffix = p)


emission calc for  ('CA-PE', 'Gross fixed capital formation, Construction') completed
CA-PE CO2e vector appended
now cal  CA-NS


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\1798386228.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_gfcfc_CO2e =  left.join(right, rsuffix = p)


emission calc for  ('CA-NS', 'Gross fixed capital formation, Construction') completed
CA-NS CO2e vector appended
now cal  CA-NB


...

C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\1798386228.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_gfcfc_CO2e =  left.join(right, rsuffix = p)


emission calc for  ('CA-NT', 'Gross fixed capital formation, Construction') completed
CA-NT CO2e vector appended
now cal  CA-NU


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\1798386228.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_gfcfc_CO2e =  left.join(right, rsuffix = p)


emission calc for  ('CA-NU', 'Gross fixed capital formation, Construction') completed
CA-NU CO2e vector appended
dataframe created successfuly


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_7832\1798386228.py:39: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca_gfcfc_CO2e =  left.join(right, rsuffix = p)


In [95]:
ca_gfcfc_CO2e.iloc[:, [5]].sum()

(CA-ON, kg CO2 eq (short))    2.682371e+10
dtype: float64

In [67]:
x=pd.DataFrame(ca_gfcfc_CO2e.sum(1))

In [70]:
x

0
CA-AB Canola (including rapeseed)                        6.945862e+06
      Oilseeds (except canola)                           1.099743e+05
      Wheat                                              4.012386e+06
      Grains (except wheat)                              2.225629e+06
      Fresh potatoes                                     7.388019e+05
...                                                               ...
WM    Membership organisation services n.e.c. (91)       1.001509e+04
      Recreational, cultural and sporting services (92)  9.599683e+04
      Other services (93)                                1.256129e+05
      Private households with employed persons (95)      7.717483e+03
      Extra-territorial organizations and bodies         0.000000e+00

[16196 rows x 1 columns]

In [72]:
x.sum()

0    8.961493e+10
dtype: float64

In [64]:
ca_gfcfc_CO2e

(CA-NL, kg CO2 eq (short))  \
CA-AB Canola (including rapeseed)                                      37957.561636   
      Oilseeds (except canola)                                           610.775767   
      Wheat                                                            39539.066328   
      Grains (except wheat)                                            16174.130032   
      Fresh potatoes                                                    4555.109947   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                       242.849112   
      Recreational, cultural and sporting services (92)                 1901.279345   
      Other services (93)                                               3066.106492   
      Private households with employed persons (95)                      171.857540   
      Extra-territorial organizations and bodies                           0.000000   

                                                         (CA-PE, kg CO2 eq (short))  \
CA-AB Canola (including rapeseed)                                       7918.865131   
      Oilseeds (except canola)                                           105.936852   
      Wheat                                                             4191.233271   
      Grains (except wheat)                                             2040.702881   
      Fresh potatoes                                                     649.141292   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                        27.758345   
      Recreational, cultural and sporting services (92)                  253.403571   
      Other services (93)                                                354.689037   
      Private households with employed persons (95)                       20.011389   
      Extra-territorial organizations and bodies                           0.000000   

                                                         (CA-NS, kg CO2 eq (short))  \
CA-AB Canola (including rapeseed)                                      51182.060397   
      Oilseeds (except canola)                                           681.144326   
      Wheat                                                            28525.743728   
      Grains (except wheat)                                            13004.147023   
      Fresh potatoes                                                    3932.065176   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                       205.773706   
      Recreational, cultural and sporting services (92)                 1634.025468   
      Other services (93)                                               2590.472665   
      Private households with employed persons (95)                      146.985483   
      Extra-territorial organizations and bodies                           0.000000   

                                                         (CA-NB, kg CO2 eq (short))  \
CA-AB Canola (including rapeseed)                                      59136.570579   
      Oilseeds (except canola)                                           682.545132   
      Wheat                                                            24408.483842   
      Grains (except wheat)                                            11691.383490   
      Fresh potatoes                                                    3272.801779   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                       159.820641   
      Recreational, cultural and sporting services (92)                 1319.271551   
      Other services (93)                                               2003.485208   
      Private households with employed persons (95) 

In [16]:
ca_gfcfc_CO2e.to_csv(expath + 'ca_gfcf_construction_co2e_green.csv')

# Sankey source emission calculation: green+blue (origin) -> blue (type)
    1.calculate each column in matrix D=(L(AY)). This would give me the origin of emissions for each community (material) 
	2. extract the  X=CS*diag(Dα1)
    combine into new D [16k*16K]
	4. Check results. If this gives the correct numbers create a for loop that calculates total output on all fo D columns indevidually


In [3]:
#list provinces and sectors to run in loop
sector = 'Gross fixed capital formation, Construction'
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
sub_sectors = tables_Y[('CA-ON', sector)]
sub_sec_list = sub_sectors.columns.values.tolist()
len(sub_sec_list) # list length is 54 sectors
#print (sub_sec_list)

54

In [4]:
# Designate final demand sector
# this demo uses ('CA-ON', 'Gross fixed capital formation, Construction')
#define province /sector /sub sector
province = provinces
sector = 'Gross fixed capital formation, Construction'
sub_sector = sub_sec_list


# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector, sub_sector)
final_demand_category

(['CA-NL',
  'CA-PE',
  'CA-NS',
  'CA-NB',
  'CA-QC',
  'CA-ON',
  'CA-MB',
  'CA-SK',
  'CA-AB',
  'CA-BC',
  'CA-YT',
  'CA-NT',
  'CA-NU'],
 'Gross fixed capital formation, Construction',
 ['Accommodation and food services',
  'Administrative and support, waste management and remediation services',
  'Animal production',
  'Arts, entertainment and recreation',
  'Beverage and tobacco product manufacturing',
  'Chemical manufacturing',
  'Clothing and leather and allied product manufacturing',
  'Computer and electronic product manufacturing',
  'Construction',
  'Crop production',
  'Defence services',
  'Educational services (private)',
  'Educational services (public)',
  'Electrical equipment, appliance and component manufacturing',
  'Fabricated metal product manufacturing',
  'Finance and insurance',
  'Fishing, hunting and trapping',
  'Food manufacturing',
  'Forestry and logging',
  'Furniture and related product manufacturing',
  'Health care and social assistance',
  'Hol

## Sankey DF loop

In [9]:
# Designate final demand sector
# this demo uses ('CA-ON', 'Gross fixed capital formation, Construction')
#define province /sector /sub sector
province = provinces
sector = 'Gross fixed capital formation, Construction'
sub_sector = sub_sec_list


# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector)
final_demand_category

NameError: name 'sub_sec_list' is not defined

In [10]:
final_demand_category = (province[5], sector, sub_sector[44]) #ON, residential sructures
print(final_demand_category)


('CA-ON', 'Gross fixed capital formation, Construction', 'Residential structures')


In [12]:
#1. extract Y
on_gfcf_sub_tablesTY = tables_Y[(final_demand_category)]
#transform series into df
on_gfcf_sub_Y = pd.DataFrame(on_gfcf_sub_Y)
on_gfcf_sub_Y

# 1.2 this line calc the total intermediate output of sector. D = (A)16Kx16K * (Y)16Kx1K
on_gfcf_sub_AY= vector_X(tables_A, on_gfcf_sub_Y)
#call diagonolised table. [0]  is the vector_X series, [1] is the diagonolized df
#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
df_D = on_gfcf_sub_AY[1]

In [ ]:
# create a loop in 1 province 
#1. extract Y
on_gfcf_sub_Y = tables_Y[(final_demand_category)]
#transform series into df
on_gfcf_sub_Y = pd.DataFrame(on_gfcf_sub_Y)
on_gfcf_sub_Y

# 1.2 this line calc the total intermediate output of sector. D = (A)16Kx16K * (Y)16Kx1K
on_gfcf_sub_AY= vector_X(tables_A, on_gfcf_sub_Y)
#call diagonolised table. [0]  is the vector_X series, [1] is the diagonolized df
#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
df_D = on_gfcf_sub_AY[1]

#FOLLY
df_D.head().sum(1)

In [65]:
# run loop on all construction sectors in province
#extract sector list from province:
final_demand_category = (province[5], sector, sub_sector[44]) #ON, residential sructures
df = tables_Y[province[5],sector]
sub_sec_list = df.columns.to_list()
sub_sec_list

['Accommodation and food services',
 'Administrative and support, waste management and remediation services',
 'Animal production',
 'Arts, entertainment and recreation',
 'Beverage and tobacco product manufacturing',
 'Chemical manufacturing',
 'Clothing and leather and allied product manufacturing',
 'Computer and electronic product manufacturing',
 'Construction',
 'Crop production',
 'Defence services',
 'Educational services (private)',
 'Educational services (public)',
 'Electrical equipment, appliance and component manufacturing',
 'Fabricated metal product manufacturing',
 'Finance and insurance',
 'Fishing, hunting and trapping',
 'Food manufacturing',
 'Forestry and logging',
 'Furniture and related product manufacturing',
 'Health care and social assistance',
 'Holding companies',
 'Hospitals',
 'Information and cultural industries',
 'Machinery manufacturing',
 'Mining and quarrying (except oil and gas)',
 'Miscellaneous manufacturing',
 'Non-metallic mineral product manufa

In [6]:
# Designate final demand sector
# this demo uses ('CA-ON', 'Gross fixed capital formation, Construction')
#define province /sector /sub sector
province = provinces
sector = 'Gross fixed capital formation, Construction'
sub_sector = sub_sec_list


# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector)
final_demand_category

NameError: name 'sub_sec_list' is not defined

In [6]:
# create a summ df
df_s_env_sum = pd.DataFrame(index = tables_A.index) #an empty environmrntal impact summary df
#create new indexed DF 
df_D_env = pd.DataFrame(index = tables_A.index)

for p in provinces[:1]:
    for s in sub_sector[:1]:
        print ('running ', p, s)
        p_gfcf_sub_Y = tables_Y[(p, sector, s)]
        p_gfcf_sub_Y = pd.DataFrame(p_gfcf_sub_Y)
        p_gfcf_sub_Y
        # 1.2 this line calc the total intermediate output of sector. D = (A)16Kx16K * (Y)16Kx1K
        p_gfcf_sub_AY= vector_X(tables_A, p_gfcf_sub_Y)
        print ('generating D table for ',p,s )
        #call diagonolised table. [0]  is the vector_X series, [1] is the diagonolized df
        #2. Calculate environmental impact. returnes kgCO2e for called total output vector  
        # call [1] for diagonal or [0] sector summary
        df_D = p_gfcf_sub_AY[1]

        # calculating each sub sector
        # list of columns in symetric table (len = 16196)
        col_d = df_D.columns.to_list()
        col_d[0]
        #create new indexed DF 
        #df_D_env = pd.DataFrame(index = df_D.index)

        # run the environmental function 
        for c in col_d[0:2]:
            # d[i] vector
            d = pd.DataFrame(df_D[c])
            d_diag = pd.DataFrame(np.diag(d.sum(1)), index = d.index, columns = d.index)
            d_diag
            # multiply CS*d[i]diag vector
            df = env_impact(tables_CS, d_diag)
            print ('In sector ', p, " ", s,' env_calc for' , i, ' excecuted')
            df_D_env = pd.concat([df_D_env, df], axis=1, sort = False)
            print ('In ', p, ' ', sector, ' ', s, i , ' added') #sector added note
            df_sum = df_D_env.sum(1)
        df_s_env_sum = pd.concat([df_s_env_sum,df_sum], axis=1, sort = False)
        
    
df_sum

#check why env df is empty after calculation compared to the col_d loop



running  CA-NL Accommodation and food services
generating D table for  CA-NL Accommodation and food services
In sector  CA-NL   Accommodation and food services  env_calc for ('CA-AB', 'Canola (including rapeseed)')  excecuted
In  CA-NL   Gross fixed capital formation, Construction   Accommodation and food services ('CA-AB', 'Canola (including rapeseed)')  added
In sector  CA-NL   Accommodation and food services  env_calc for ('CA-AB', 'Oilseeds (except canola)')  excecuted
In  CA-NL   Gross fixed capital formation, Construction   Accommodation and food services ('CA-AB', 'Oilseeds (except canola)')  added
CA-AB  Canola (including rapeseed)                          0.0
       Oilseeds (except canola)                             0.0
       Wheat                                                0.0
       Grains (except wheat)                                0.0
       Fresh potatoes                                       0.0
                                                           ... 
WM 

KeyboardInterrupt: 

In [34]:
print (len(sub_sector))
sub_sector[44]

54


'Residential structures'

In [27]:
df_D_env.describe()

,"(Climate change, short term, kg CO2 eq (short))","(Climate change, short term, kg CO2 eq (short))"
count,16196.0,16196.0
mean,0.0,0.0
std,0.0,0.0
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


In [23]:
df= tables_Y[(provinces[1], sector, sub_sector[1])]
df

CA-AB  Canola (including rapeseed)                          0.0
       Oilseeds (except canola)                             0.0
       Wheat                                                0.0
       Grains (except wheat)                                0.0
       Fresh potatoes                                       0.0
                                                           ... 
WM     Membership organisation services n.e.c. (91)         0.0
       Recreational, cultural and sporting services (92)    0.0
       Other services (93)                                  0.0
       Private households with employed persons (95)        0.0
       Extra-territorial organizations and bodies           0.0
Name: (CA-PE, Gross fixed capital formation, Construction, Administrative and support, waste management and remediation services), Length: 16196, dtype: float64

In [85]:
df_sum

CA-AB  Canola (including rapeseed)                          0.0
       Oilseeds (except canola)                             0.0
       Wheat                                                0.0
       Grains (except wheat)                                0.0
       Fresh potatoes                                       0.0
                                                           ... 
WM     Membership organisation services n.e.c. (91)         0.0
       Recreational, cultural and sporting services (92)    0.0
       Other services (93)                                  0.0
       Private households with employed persons (95)        0.0
       Extra-territorial organizations and bodies           0.0
Length: 16196, dtype: float64

In [59]:
df_D.head()

CA-AB  \
                                  Canola (including rapeseed)   
CA-AB Canola (including rapeseed)                    3.901243   
      Oilseeds (except canola)                       0.000000   
      Wheat                                          0.000000   
      Grains (except wheat)                          0.000000   
      Fresh potatoes                                 0.000000   

                                                                        \
                                  Oilseeds (except canola)       Wheat   
CA-AB Canola (including rapeseed)                 0.000000    0.000000   
      Oilseeds (except canola)                    1.635234    0.000000   
      Wheat                                       0.000000  373.380649   
      Grains (except wheat)                       0.000000    0.000000   
      Fresh potatoes                              0.000000    0.000000   

                                                                        \
                                  Grains (except wheat) Fresh potatoes   
CA-AB Canola (including rapeseed)              0.000000       0.000000   
      Oilseeds (except canola)                 0.000000       0.000000   
      Wheat                                    0.000000       0.000000   
      Grains (except wheat)                   92.519801       0.000000   
      Fresh potatoes                           0.000000      25.837858   

                                                        \
                                  Fresh fruit and nuts   
CA-AB Canola (including rapeseed)                  0.0   
      Oilseeds (except canola)                     0.0   
      Wheat                                        0.0   
      Grains (except wheat)                        0.0   
      Fresh potatoes                               0.0   

                                                                     \
                                  Other miscellaneous crop products   
CA-AB Canola (including rapeseed)                               0.0   
      Oilseeds (except canola)                                  0.0   
      Wheat                                                     0.0   
      Grains (except wheat)                                     0.0   
      Fresh potatoes                                            0.0   

                                                                      \
                                  Fresh vegetables (except potatoes)   
CA-AB Canola (including rapeseed)                                0.0   
      Oilseeds (except canola)                                   0.0   
      Wheat                                                      0.0   
      Grains (except wheat)                                      0.0   
      Fresh potatoes                                             0.0   

                                                                                           \
                                  Imputed feed (animal feed produced for own consumption)   
CA-AB Canola (including rapeseed)                                                0.0        
      Oilseeds (except canola)                                                   0.0        
      Wheat                                                                      0.0        
      Grains (except wheat)                                                      0.0        
      Fresh potatoes                                                             0.0        

                                                                                                \
                                  Floriculture and nursery products (except cannabis), and sod   
CA-AB Canola (including rapeseed)                                                0.0             
      Oilseeds (except canola)                                                   0.0             
      Wheat                                                                      0.0             
      Grai

In [6]:
col_d = df_D.columns.to_list()
col_d[0]
len(col_d)
col_d

[('CA-AB', 'Canola (including rapeseed)'),
 ('CA-AB', 'Oilseeds (except canola)'),
 ('CA-AB', 'Wheat'),
 ('CA-AB', 'Grains (except wheat)'),
 ('CA-AB', 'Fresh potatoes'),
 ('CA-AB', 'Fresh fruit and nuts'),
 ('CA-AB', 'Other miscellaneous crop products'),
 ('CA-AB', 'Fresh vegetables (except potatoes)'),
 ('CA-AB', 'Imputed feed (animal feed produced for own consumption)'),
 ('CA-AB', 'Floriculture and nursery products (except cannabis), and sod'),
 ('CA-AB', 'Cannabis plants, seeds and flowering tops'),
 ('CA-AB', 'Cattle and calves'),
 ('CA-AB', 'Unprocessed milk'),
 ('CA-AB', 'Hogs'),
 ('CA-AB', 'Eggs in shell'),
 ('CA-AB', 'Poultry'),
 ('CA-AB', 'Other live animals'),
 ('CA-AB', 'Raw furskins, and animal products n.e.c.'),
 ('CA-AB', 'Imputed fertilizer (fertilizer produced for own consumption)'),
 ('CA-AB', 'Logs and bolts'),
 ('CA-AB', 'Pulpwood'),
 ('CA-AB', 'Fuel wood'),
 ('CA-AB', 'Rough untreated poles, posts and piling'),
 ('CA-AB', 'Fish, crustaceans, shellfish and other fi

In [13]:
# list of columns in symetric table (len = 16196)
col_d = df_D.columns.to_list()
col_d[0]
#create new indexed DF 
df_D_env = pd.DataFrame(index = df_D.index)



# run the environmental function 
for i in col_d[0:5]:
    # d[i] vector
    d = pd.DataFrame(df_D[i])
    d_diag = pd.DataFrame(np.diag(d.sum(1)), index = d.index, columns = d.index)
    d_diag
    # multiply CS*d[i]diag vector
    df = env_impact(tables_CS, d_diag)
    print ('In sector S, env_calc for' , i, ' excecuted')
    df_D_env = pd.concat([df_D_env, df], axis=1, sort = False)
    
df_sum = df_D_env.sum(1)
print ('In sector S, ',i , ' added') #sector added note
df_sum


In sector S, env_calc for ('CA-AB', 'Canola (including rapeseed)')  excecuted
In sector S, env_calc for ('CA-AB', 'Oilseeds (except canola)')  excecuted
In sector S, env_calc for ('CA-AB', 'Wheat')  excecuted
In sector S, env_calc for ('CA-AB', 'Grains (except wheat)')  excecuted
In sector S, env_calc for ('CA-AB', 'Fresh potatoes')  excecuted
In sector S,  ('CA-AB', 'Fresh potatoes')  added


CA-AB  Canola (including rapeseed)                            3.156338
       Oilseeds (except canola)                               1.256821
       Wheat                                                302.145063
       Grains (except wheat)                                 75.355667
       Fresh potatoes                                        20.851597
                                                               ...    
WM     Membership organisation services n.e.c. (91)           0.000000
       Recreational, cultural and sporting services (92)      0.000000
       Other services (93)                                    0.000000
       Private households with employed persons (95)          0.000000
       Extra-territorial organizations and bodies             0.000000
Length: 16196, dtype: float64

In [46]:
type(df_sum)

pandas.core.series.Series

In [21]:
# d[i] vector
d = pd.DataFrame(df_D[col_d[0]])
d_diag = pd.DataFrame(np.diag(d.sum(1)), index = d.index, columns = d.index)
d_diag
# multiply CS*d[i]diag vector
env_d = env_impact(tables_CS, d_diag)
type(env_d)

CA-AB  \
                                                        Canola (including rapeseed)   
CA-AB Canola (including rapeseed)                                          3.901243   
      Oilseeds (except canola)                                             0.000000   
      Wheat                                                                0.000000   
      Grains (except wheat)                                                0.000000   
      Fresh potatoes                                                       0.000000   
...                                                                             ...   
WM    Membership organisation services n.e.c. (91)                         0.000000   
      Recreational, cultural and sporting services (92)                    0.000000   
      Other services (93)                                                  0.000000   
      Private households with employed persons (95)                        0.000000   
      Extra-territorial organizations and bodies                           0.000000   

                                                                                  \
                                                        Oilseeds (except canola)   
CA-AB Canola (including rapeseed)                                            0.0   
      Oilseeds (except canola)                                               0.0   
      Wheat                                                                  0.0   
      Grains (except wheat)                                                  0.0   
      Fresh potatoes                                                         0.0   
...                                                                          ...   
WM    Membership organisation services n.e.c. (91)                           0.0   
      Recreational, cultural and sporting services (92)                      0.0   
      Other services (93)                                                    0.0   
      Private households with employed persons (95)                          0.0   
      Extra-territorial organizations and bodies                             0.0   

                                                               \
                                                        Wheat   
CA-AB Canola (including rapeseed)                         0.0   
      Oilseeds (except canola)                            0.0   
      Wheat                                               0.0   
      Grains (except wheat)                               0.0   
      Fresh potatoes                                      0.0   
...                                                       ...   
WM    Membership organisation services n.e.c. (91)        0.0   
      Recreational, cultural and sporting services (92)   0.0   
      Other services (93)                                 0.0   
      Private households with employed persons (95)       0.0   
      Extra-territorial organizations and bodies          0.0   

                                                                               \
                                                        Grains (except wheat)   
CA-AB Canola (including rapeseed)                                         0.0   
      Oilseeds (except canola)                                            0.0   
      Wheat                                                               0.0   
      Grains (except wheat)                                               0.0   
      Fresh potatoes                                                      0.0   
...                                                                       ...   
WM    Membership organisation services n.e.c. (91)                        0.0   
      Recreational, cultural and sporting services (92)                   0.0   
      Other services (93)                                                 0.0   
      Private households with employed persons (95)                       0.0   
      Extra-territorial organizatio

In [25]:
# multiply CS*d[i]diag vector
env_d = env_impact(tables_CS, d_diag)
env_d


CA-AB  Canola (including rapeseed)                                         3.156338
PT     Crops nec                                                           0.000000
PL     Wood waste for treatment: landfill                                  0.000000
       Membership organisation services n.e.c. (91)                        0.000000
       Recreational, cultural and sporting services (92)                   0.000000
                                                                             ...   
CA-QC  Community college and C.E.G.E.P services provided by governments    0.000000
       University services provided by governments                         0.000000
       Other educational services provided by governments                  0.000000
       Hospital services provided by governments                           0.000000
WM     Extra-territorial organizations and bodies                          0.000000
Name: (Climate change, short term, kg CO2 eq (short)), Length: 16196, dtype:

In [37]:
# calculate CSD
d_master_green = pd.DataFrame()

for d in col_d[0]:
    # create a d vector of commodity
    df = df_D[d]
    on_gfcf_sub_CSX = env_impact(tables_CS, df)
    left = d_master_green
    right = on_gfcf_sub_CSX
    d_master_green = left.join(right, rsuffix = d)


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_5568\1684017000.py:10: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  d_master_green = left.join(right, rsuffix = d)


KeyError: 'Canola (including rapeseed)'

In [ ]:
# create lists to stor data
ca_master_gfcfc_df_green = pd.DataFrame()
province_gfcfc_list = []

for p in provinces[0]:
    for s in sub_sector[0:1]:
        final_demand_category = (p, sector, s)
        #print ('Caclc ' + str(final_demand_category))
        #print (tables_Y[(p, sector, s)].sum())
        #---- calculationcode goes here ----
         # isolate final demnad vector

        on_gfcf_sub_Y = tables_Y[(final_demand_category)]
        #transform series into df
        on_gfcf_sub_Y = pd.DataFrame(on_gfcf_sub_Y)
        on_gfcf_sub_Y
        print ('in ',p, ' claculating ', s)
        # 1.2 this line calc the total intermediate output of sector. D = (A)16Kx16K * (Y)16Kx1K
        on_gfcf_sub_AY= vector_X(tables_A, on_gfcf_sub_Y)
        #call diagonolised table. [0]  is the vector_X series, [1] is the diagonolized df

        #2. Calculate environmental impact. returnes kgCO2e for called total output vector  
        # call [1] for diagonal or [0] sector summary
        col_d = on_gfcf_sub_AY.columns.to_list()
        
        for d in col_d:
        on_gfcf_sub_CSX = env_impact(tables_CS, on_gfcf_sub_AY[1])

        #3. assign title to column
        on_gfcf_sub_CSX = pd.DataFrame(on_gfcf_sub_CSD).rename(
            columns = {'Climate change, short term': (p+'_gfcfc_'+ s)})
        on_gfcf_sub_CSX


# Total economy emissions

In [7]:
province = 'CA-ON'
sector = 'Gross fixed capital formation, Construction'
sub_sector = 'Residential structures'
# this parameter will propegate as a headar to the env impact table
final_demand_category = (province)
final_demand_category

#This line findes the final demand category to analyze (province, sector, sub sector)
on_final_prov = tables_Y[(final_demand_category)]
# transformation to df requiered for dot() multiplication
on_final_prov = pd.DataFrame(on_final_prov)
on_final_prov

Changes in inventories  \
                                                        Changes in inventories, finished goods and goods in process   
CA-AB Canola (including rapeseed)                                                             27319.218321            
      Oilseeds (except canola)                                                                    0.000000            
      Wheat                                                                                       0.000000            
      Grains (except wheat)                                                                       0.000000            
      Fresh potatoes                                                                              0.000000            
...                                                                                                    ...            
WM    Membership organisation services n.e.c. (91)                                                0.000000            
      Recreational, cultural and sporting services (92)                                         122.620657            
      Other services (93)                                                                         0.000000            
      Private households with employed persons (95)                                               0.000000            
      Extra-territorial organizations and bodies                                                  0.000000            

                                                                                                                              \
                                                        Changes in inventories, raw materials and goods purchased for resale   
CA-AB Canola (including rapeseed)                                                             0.000000e+00                     
      Oilseeds (except canola)                                                                0.000000e+00                     
      Wheat                                                                                   3.961778e+06                     
      Grains (except wheat)                                                                   4.682439e+01                     
      Fresh potatoes                                                                          0.000000e+00                     
...                                                                                                    ...                     
WM    Membership organisation services n.e.c. (91)                                            0.000000e+00                     
      Recreational, cultural and sporting services (92)                                       0.000000e+00                     
      Other services (93)                                                                     0.000000e+00                     
      Private households with employed persons (95)                                           0.000000e+00                     
      Extra-territorial organizations and bodies                                              0.000000e+00                     

                                                        Governments final consumption expenditure  \
                                                                                 Defence services   
CA-AB Canola (including rapeseed)                                                             0.0   
      Oilseeds (except canola)                                                                0.0   
      Wheat                                                                                   0.0   
      Grains (except wheat)                                                                   0.0   
      Fresh potatoes                                                                          0.0   
...                                                                                           ...   
WM    Membership organisation services n.e.c. (91)                      

In [10]:
on_final_prov.sum()

Changes in inventories                                                     Changes in inventories, finished goods and goods in process             1.526320e+09
                                                                           Changes in inventories, raw materials and goods purchased for resale    3.492112e+09
Governments final consumption expenditure                                  Defence services                                                        9.196179e+09
                                                                           Educational services                                                    3.289863e+10
                                                                           Hospitals                                                               3.052633e+10
                                                                                                                                                       ...     
Household final consumption expenditure 

In [11]:
# 1. this line calc total output of sector. X = (L)16Kx16K * (Y)16Kx1
prov_X = vector_X(tables_L, tables_Y)
#call diagonolised X table. [0]  is the vector_X series, [1] is the diagonolized df

In [16]:
prov_X[0].sum()

Changes in inventories                                                     Changes in inventories, finished goods and goods in process             3.621203e+09
                                                                           Changes in inventories, raw materials and goods purchased for resale    1.402515e+10
Governments final consumption expenditure                                  Defence services                                                        1.459603e+10
                                                                           Educational services                                                    4.034905e+10
                                                                           Hospitals                                                               4.937187e+10
                                                                                                                                                       ...     
Household final consumption expenditure 

In [18]:
#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
prov_CO2e = env_impact(tables_CS, prov_X[0])

In [22]:
#3. assign title to column  
prov_CO2e = pd.DataFrame(prov_CO2e).rename(
    columns = {'Climate change, short term': final_demand_category})
prov_CO2e.sum()
prov_CO2e

,,CA-ON
,,kg CO2 eq (short)
International exports,Exports,1.201184e+11
Household final consumption expenditure,Food,2.703695e+10
International exports,Re-exports,1.409474e+10
"Gross fixed capital formation, Construction",Residential structures,1.345435e+10
Household final consumption expenditure,Fuels and lubricants,1.240944e+10
...,...,...
"Gross fixed capital formation, Intellectual property products",Oil sands extraction,0.000000e+00
"Gross fixed capital formation, Construction",Oil sands extraction,0.000000e+00
Household final consumption expenditure,Expenditure by Canadians residing in other provinces or territories,-1.514761e+09


In [23]:
prov_CO2e.sum()

CA-ON  kg CO2 eq (short)    3.248222e+11
dtype: float64

### For graph analysis by emission source by country

In [125]:
#imports process emission DF of GFCFC Canada by province (green) and renames coulmns.
ca_gfcf_construction = pd.read_csv(expath + "ca_gfcf_construction_co2e.csv")
ca_gfcf_construction= ca_gfcf_construction.rename(columns = {	'Unnamed: 0':'country', 'Unnamed: 1':'commodity'})
ca_gfcf_construction
#summs total emisssions in of Canada in a anew column and groups it by country
ca_gfcf_construction['CAN_total'] = ca_gfcf_construction.iloc[:, 2:].sum(axis=1)
ca_gfcfc_by_country = ca_gfcf_construction.groupby('country').agg({'CAN_total':np.sum})
ca_gfcfc_by_country

,CAN_total
country,
AT,5.843007e+07
AU,2.584724e+08
BE,7.197758e+07
BG,1.904476e+07
BR,5.997097e+08
...,...
WE,2.845101e+08
WF,4.047469e+08
WL,9.917938e+08


In [126]:
# validation test  - total construction in CA should be approx 89MT
ca_gfcfc_by_country.sum()

CAN_total    8.961493e+10
dtype: float64

In [129]:
ca_gfcfc_by_country.to_csv(expath + exports +"ca_gfcf_by_country_green.csv")

# TESTS

In [16]:
# All CA sll sectors all commodities run

#list provinces and sectors to run in loop
sector = 'Gross fixed capital formation, Construction'
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
sub_sectors = tables_Y[('CA-ON', sector)]
sub_sec_list = sub_sectors.columns.values.tolist()
len(sub_sec_list) # list length is 54 sectors
#print (sub_sec_list)

for p in provinces:
    for s in sub_sectors:
        print ('running ', p, s)
        on_gfcf_sub_Y = tables_Y[(p, sector, s)]
        #on_gfcf_sub_Y = pd.DataFrame(on_gfcf_sub_Y)
        #on_gfcf_sub_Y
        col_d = tables_A.columns.to_list()
        for c in col_d:
            print (p,s,c,)
on_gfcf_sub_Y

running  CA-NL Accommodation and food services
CA-NL Accommodation and food services CA-AB
CA-NL Accommodation and food services Oilseeds (except canola)
running  CA-NL Administrative and support, waste management and remediation services
CA-NL Administrative and support, waste management and remediation services CA-AB
CA-NL Administrative and support, waste management and remediation services Oilseeds (except canola)
running  CA-NL Animal production
CA-NL Animal production CA-AB
CA-NL Animal production Oilseeds (except canola)
running  CA-NL Arts, entertainment and recreation
CA-NL Arts, entertainment and recreation CA-AB
CA-NL Arts, entertainment and recreation Oilseeds (except canola)
running  CA-NL Beverage and tobacco product manufacturing
CA-NL Beverage and tobacco product manufacturing CA-AB
CA-NL Beverage and tobacco product manufacturing Oilseeds (except canola)
running  CA-NL Chemical manufacturing
CA-NL Chemical manufacturing CA-AB
CA-NL Chemical manufacturing Oilseeds (exce

CA-AB  Canola (including rapeseed)                          0.0
       Oilseeds (except canola)                             0.0
       Wheat                                                0.0
       Grains (except wheat)                                0.0
       Fresh potatoes                                       0.0
                                                           ... 
WM     Membership organisation services n.e.c. (91)         0.0
       Recreational, cultural and sporting services (92)    0.0
       Other services (93)                                  0.0
       Private households with employed persons (95)        0.0
       Extra-territorial organizations and bodies           0.0
Name: (CA-NU, Gross fixed capital formation, Construction, Wood product manufacturing), Length: 16196, dtype: float64